In [79]:
import csv
import pandas as pd
import numpy as np

还是要先把enhancer文件整好。

In [92]:
eori = pd.read_csv("emp.csv",header = 0)
esele = pd.read_csv("NTROC/fwd_avr_f1.csv",header = 0)
eori = eori[['Unnamed: 0','chr', 'start', 'end']]
eori.columns = ['id','chr', 'start', 'end']
sele = list(esele['Unnamed: 0'])

In [93]:
eoris = eori['id'].apply(lambda x: True if x in sele else False)
eoris2 = eori[eoris]

In [95]:
eoris2['name']=eoris2['id'].apply(lambda x: 'Enh'+str(x+1).zfill(6))
eoris2 = eoris2[['name','chr','start','end']]

<ipython-input-95-0bc0923280e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eoris2['name']=eoris2['id'].apply(lambda x: 'Enh'+str(x+1).zfill(6))


In [97]:
eoris2.to_csv('enhancers_f1.csv')

In [96]:
eoris2

,name,chr,start,end
16,Enh000017,chr1,3053410,3053878
17,Enh000018,chr1,3054529,3055251
18,Enh000019,chr1,3056107,3056369
19,Enh000020,chr1,3057074,3057556
20,Enh000021,chr1,3057627,3057951
...,...,...,...,...
306639,Enh306640,chrY,90824038,90825836
306640,Enh306641,chrY,90825040,90825586
306641,Enh306642,chrY,90826119,90826671
306642,Enh306643,chrY,90828152,90829213


In [83]:
e1 = e1[['chr']]

In [89]:
e0=e0.drop(columns=['Unnamed: 0'])

In [91]:
e=e1.join(e0)
e

,chr,name,start,end
0,chr1,Enh000017,3053410.0,3053878.0
1,chr1,Enh000018,3054529.0,3055251.0
2,chr1,Enh000019,3056107.0,3056369.0
3,chr1,Enh000020,3057074.0,3057556.0
4,chr1,Enh000021,3057627.0,3057951.0
...,...,...,...,...
306639,chrY,NaN,NaN,NaN
306640,chrY,NaN,NaN,NaN
306641,chrY,NaN,NaN,NaN
306642,chrY,NaN,NaN,NaN


然后开始选基因了

先弄一份简单的基因表吧

In [49]:
f = open('source/gencode.vM17.annotation_table.txt','r')
lines = f.readlines()
fn = open('vM17_geneannotation.csv','w',newline = '')
writer = csv.writer(fn)

In [50]:
writer.writerow(['chr','start','end','gene_id','gene_type','gene_name'])
for i in lines:
    if '#' in i:continue
    k = i.split('\t')
    if k[2] != 'gene':continue
    chrom = k[0]
    start = k[3]
    end = k[4]
    anno = k[8].split(';')
    idd = (anno[0].split('='))[1]
    gene_type = (anno[2].split('='))[1]
    gene_name = (anno[3].split('='))[1]
    writer.writerow([chrom,start,end,idd,gene_type,gene_name])

In [70]:
f2 = open('source/gene_TPM_Corrected.txt','r')
lines2 = f2.readlines()
gdf = pd.read_csv('vM17_geneannotation.csv',header = 0)

In [71]:
for i in lines2:
    i=i.split(' ')
    if i[0] == 'MII-Oocyte':continue
    t = gdf.loc[gdf['gene_id'] == i[0]]
    if t.empty == True: 
        print("wow这是我妹想到的")
        continue
    gdf.loc[t.index,'MII-Oocyte']=i[1]
    gdf.loc[t.index,'2-cell']=i[2]
    gdf.loc[t.index,'4-cell']=i[3]
    gdf.loc[t.index,'8-cell']=i[4]
    gdf.loc[t.index,'morula']=i[5]
    gdf.loc[t.index,'ICM']=i[6]
    gdf.loc[t.index,'TE']=i[7]
    gdf.loc[t.index,'E6.5_Epi']=i[8]
    gdf.loc[t.index,'E6.5_Exe']=i[9]

In [159]:
gdf.to_csv('vM17_geneannotation.csv')

In [189]:
fn1 = open('vM17_geneannotation.csv','r')
reader = csv.reader(fn1)
dfe = pd.read_csv('NTROC/enh_TPM.csv',index_col = 2)

In [190]:
gdf

,chr,start,end,gene_id,gene_type,gene_name,MII-Oocyte,2-cell,4-cell,8-cell,morula,ICM,TE,E6.5_Epi,E6.5_Exe
0,chr1,3073253,3074322,ENSMUSG00000102693.1,TEC,RP23-271O17.1,0,0.693616,0.1151615,0,0,0,0,0,0\n
1,chr1,3102016,3102125,ENSMUSG00000064842.1,snRNA,Gm26206,0,0,0,0,0,0,0,0,0\n
2,chr1,3205901,3671498,ENSMUSG00000051951.5,protein_coding,Xkr4,0.371837,0.05008275,0.006557,0.0271993333333333,0,0.345639,0.232552,0.879696,0.0356115\n
3,chr1,3252757,3253236,ENSMUSG00000102851.1,processed_pseudogene,RP23-317L18.1,0,0,0,0,0,0,0,0,0\n
4,chr1,3365731,3368549,ENSMUSG00000103377.1,TEC,RP23-317L18.4,0,0,0,0,0,0,0,1.265191,0.067764\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53710,chrM,13552,14070,ENSMUSG00000064368.1,protein_coding,mt-Nd6,1579.045044,2150.13888575,4128.58685325,2201.24882,2423.652832,930.7703705,1301.00387575,1375.1036985,2326.589844\n
53711,chrM,14071,14139,ENSMUSG00000064369.1,Mt_tRNA,mt-Te,192.9560855,207.2879715,386.206505,286.122121,170.65744,47.68229225,98.070473,55.2607155,72.009014\n
53712,chrM,14145,15288,ENSMUSG00000064370.1,protein_coding,mt-Cytb,2556.612793,7992.33447275,9628.45080575,10895.0188803333,6458.2937015,5756.882202,6578.697754,4817.422363,4849.1005855\n
53713,chrM,15289,15355,ENSMUSG00000064371.1,Mt_tRNA,mt-Tt,91.7873995,112.74412725,180.82487325,83.4294113333333,99.118294,7.13176275,14.6729295,31.6496515,30.894643\n


In [191]:
fn = open('enh_gene_r_r0.6.csv','w',newline='')
writer = csv.writer(fn)
writer.writerow(['gene_name','chr','gene_id','gene_loc1','gene_loc2','enhid','enh_loc1','enh_loc2','r','gene2C/ICM','geneICM/2C','enh2C/ICM','enhICM/2C','enhTPM','geneTPM'])

126

In [192]:
dfe=dfe.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
dfe

,chr,start,end,avr_MII-Oocyte,avr_2-cell,avr_4-cell,avr_8-cell,avr_morula,avr_ICM,avr_TE,avr_E6.5Epi,avr_E6.5Exe
name,,,,,,,,,,,,
Enh000017,chr1,3053410,3053878,0.000000,0.000000,2.491477,4.984360,13.024934,19.652384,6.572173,0.000000,0.000000
Enh000018,chr1,3054529,3055251,0.000000,0.228187,77.680240,242.775999,197.701179,1022.181296,462.751098,0.000000,3.051206
Enh000019,chr1,3056107,3056369,0.000000,0.000000,3.115297,25.438173,11.632956,44.678146,42.556077,0.000000,4.204142
Enh000020,chr1,3057074,3057556,0.000000,0.000000,14.272750,51.161337,28.454887,130.679826,92.528566,0.000000,123.402907
Enh000021,chr1,3057627,3057951,93.094512,4.322167,8.637119,93.595202,56.441381,87.740945,110.357745,3.276248,37.396103
...,...,...,...,...,...,...,...,...,...,...,...,...
Enh306640,chrY,90824038,90825836,0.000000,0.320705,39.382598,167.095734,185.597341,159.721474,121.418071,219.395310,151.377639
Enh306641,chrY,90825040,90825586,0.000000,0.150871,104.453381,434.493771,452.709491,411.117657,276.254267,506.612423,365.346100
Enh306642,chrY,90826119,90826671,0.000000,0.000000,23.130111,115.305890,181.747380,115.286216,100.993588,162.494802,93.785878


In [193]:
dfe

,chr,start,end,avr_MII-Oocyte,avr_2-cell,avr_4-cell,avr_8-cell,avr_morula,avr_ICM,avr_TE,avr_E6.5Epi,avr_E6.5Exe
name,,,,,,,,,,,,
Enh000017,chr1,3053410,3053878,0.000000,0.000000,2.491477,4.984360,13.024934,19.652384,6.572173,0.000000,0.000000
Enh000018,chr1,3054529,3055251,0.000000,0.228187,77.680240,242.775999,197.701179,1022.181296,462.751098,0.000000,3.051206
Enh000019,chr1,3056107,3056369,0.000000,0.000000,3.115297,25.438173,11.632956,44.678146,42.556077,0.000000,4.204142
Enh000020,chr1,3057074,3057556,0.000000,0.000000,14.272750,51.161337,28.454887,130.679826,92.528566,0.000000,123.402907
Enh000021,chr1,3057627,3057951,93.094512,4.322167,8.637119,93.595202,56.441381,87.740945,110.357745,3.276248,37.396103
...,...,...,...,...,...,...,...,...,...,...,...,...
Enh306640,chrY,90824038,90825836,0.000000,0.320705,39.382598,167.095734,185.597341,159.721474,121.418071,219.395310,151.377639
Enh306641,chrY,90825040,90825586,0.000000,0.150871,104.453381,434.493771,452.709491,411.117657,276.254267,506.612423,365.346100
Enh306642,chrY,90826119,90826671,0.000000,0.000000,23.130111,115.305890,181.747380,115.286216,100.993588,162.494802,93.785878


In [194]:
def find_enhancer(chrg,geneloc1,geneloc2,fe):
    fn2 = open(fe,'r')
    readere = csv.reader(fn2)
    enhancerlist=[]
    for enh in readere:
        if enh[2] == 'name' :continue
        if enh[3]==chrg and int(enh[4])>=int(geneloc1)-1000000 and int(enh[5])<=int(geneloc2)+1000000 :
            enhancerlist.append(enh[2])
    return enhancerlist

In [195]:
def TPM_r(genetpm,enhtpm):
    data=pd.DataFrame({'e':enhtpm,'g':genetpm})
    datar=data.corr()
    if(np.isnan(datar.iloc[0,1])) :
        r = -1
    else:
        r=datar.iloc[0,1]
    if genetpm[1] == 0:gfold1=0
    elif genetpm[5] == 0 :gfold1 = 10000
    else: gfold1 =genetpm[1]/genetpm[5]
    if genetpm[5] == 0:gfold2=0
    elif genetpm[1] == 0 :gfold2= 10000
    else: gfold2 =genetpm[5]/genetpm[1]  
    if enhtpm[1] == 0:efold1=0
    elif enhtpm[5] == 0 :efold1= 10000
    else: efold1 =enhtpm[1]/enhtpm[5]
    if enhtpm[5] == 0:efold2=0
    elif enhtpm[1] == 0 :efold2= 10000
    else: efold2 =enhtpm[5]/enhtpm[1] 
    return [r,gfold1,gfold2,efold1,efold2,genetpm,enhtpm]    

In [196]:
for genei in reader:
    if genei[1] == 'chr': continue
    chrg=genei[1]
    geneloc1=genei[2]
    geneloc2=genei[3]
    geneid = genei[4]
    genename = genei[6]
    elist = find_enhancer(chrg,geneloc1,geneloc2,'NTROC/enh_TPM.csv')
    gtpm = [float(genei[7]),float(genei[8]),float(genei[9]),float(genei[10]),float(genei[11]),float(genei[12]),float(genei[13]),float(genei[14]),float(genei[15])]
    for enhi in elist:
        etpm=list(dfe.loc[enhi,'avr_MII-Oocyte':'avr_E6.5Exe'])
        enhs = dfe.loc[enhi,'start']
        enhe = dfe.loc[enhi,'end']
        r = TPM_r(gtpm,etpm)
        if (r[0] == -1) | (abs(r[0])<0.6):continue
        else:
            writer.writerow([genename,chrg,geneid,geneloc1,geneloc2,enhi,enhs,enhe,r[0],r[1],r[2],r[3],r[4],etpm,gtpm])

In [59]:
gdf.loc[t.index]

,chr,start,end,gene_id,gene_type,gene_name
2,chr1,3205901,3671498,ENSMUSG00000051951.5,protein_coding,Xkr4


In [66]:
k=[4,5]

In [67]:
gdf['TPM'] = k

ValueError: Length of values (2) does not match length of index (53715)